In [1]:
from ediblepickle import checkpoint
import os

cache_dir = 'cache'
if not os.path.exists(cache_dir):
    os.mkdir(cache_dir)

#@checkpoint(key=lambda args, kwargs: args[0] + '.p', work_dir=cache_dir)
def fetch_weekly_theatre_stats(start_day):
    url_ = "http://www.broadwayworld.com/grosses.cfm"
    import requests
    result = requests.post(url_, params = {'days':start_day})
    return result.content

def all_available_dates():
    from pyquery import PyQuery
    url_ = "http://www.broadwayworld.com/grosses.cfm"
    import requests
    result = requests.post(url_, params = {'days':'2016-03-06'}) # a random day
    dom=PyQuery(result.content)
    return sorted(dom.find('select').find('option').map(lambda op: this.text))

all_dates = all_available_dates()
dates_of_interest = [datestr for datestr in all_dates if int(datestr[:4])>=2009]

In [7]:
from bs4 import BeautifulSoup
def td_texts(html_str):
    soup = BeautifulSoup(html_str, 'html.parser')
    return [col.get_text().strip() for col in soup.find_all('td')]

global _row
def scrape_show_stats(dom):
    show_stats = []
    header_row = dom.find("table TR[style *= 'background-color:black']")
    table_rows = dom.find("table TR[onmouseover *= '#FFFF99']")
    for row_ in table_rows:
        _row = PyQuery(row_)
        if _row.find('Td') is None:
            continue
        show_name = _row.find('Td:first').find("a[href *= '/grosses/']")
        theatre_name = _row.find('Td:first').find("a[href *= '/grossestheater/']")
        if len(_row.find('Td:first').find("a[href *= '/grosses/']")) > 0:
            if len(_row.find('Td:first').find("a[href *= '/grossestheater/']")) > 0:
                try:
                    ro_info = dict(zip(td_texts(header_row.html()), td_texts(_row.html())))
                    ro_info['Show'] = show_name.text()
                    ro_info['Theatre'] = theatre_name.text()
                    show_stats.append(ro_info)
                except TypeError as e:
                    pass
    return show_stats

In [ ]:
len(dat)

### Scraping MainLoop below

In [ ]:
from time import sleep
from pyquery import PyQuery
htmls = {}
dat = {}
for date_ in dates_of_interest:
    all_html = fetch_weekly_theatre_stats(date_)
    htmls[date_] = all_html
    dat[date_] = scrape_show_stats(PyQuery(all_html))
    sleep(20)

### Changiong string values to integer or float

In [ ]:
def clean_var(st):
    if (type(st) is not unicode) and (type(st) is not str):
        return st
    st2=st.replace("%","").replace("$","").replace(",","")
    try:
        tmp = float(st2)
    except Exception as e:
        return st
    if abs(tmp%1) < .00000001:
        return int(tmp)
    return tmp

def clean_theatre_doc(doc):
    for k,v in doc.items():
        if k in ['Theatre', 'Show']:
            continue
        doc[k] = clean_var(v)
    return doc

### Saving the result to disk using dill

In [ ]:
import dill
dill.dump(dat, open('all_theatre_scraped2009_to_2017.pkd', 'w'))

In [ ]:
import dill
dat = dill.load(open('all_theatre_scraped2009_to_2017.pkd', 'r'))

In [ ]:
# more data cleaning
for date_,docs in dat.items():
    for doc in docs:
         try:
            doc['Theatre'] = doc['Theatre'].replace(".","").lower()
            doc["_id"] =date_ +"_"+ doc['Theatre'].title()
            doc["date"]=date_
            doc["Show"]=doc["Show"].lower()
            save_doc(clean_theatre_doc(doc))  # save_doc to be implemented
        except Exception as e:
            print e
            print doc